<div style="text-align: center">
    <font size="6">
        <b>
            Algorithms 1: Divide and Conquer
        </b>
    </font>
</div>


In [1]:
import copy
import numpy as np
from time import process_time
import math

# 1. Integer Multiplication

In [2]:
x = 5678
y = 1234
x*y

7006652

### Simple Loop

In [3]:
def simple_loop_mult(x,y):
    minnum = min(x,y)
    maxnum = max(x,y)
    s=0
    for i in range (minnum):
        s+= maxnum
    return s

In [4]:
simple_loop_mult(x,y)

7006652

This algorithm is ridiculously bad, as the it's time complexity is $\mathcal{O}(m)$ where m is minimum of both numbers, and not the length of the number.

### Grade School Algorithm

In [5]:
def grade_school_mult(n1,n2):
    n1list = [int(somenum) for somenum in str(n1)]
    n2list = [int(somenum) for somenum in str(n2)]
    
    final_res = 0
    for i in range(len(n1list)):
        n1dig = n1list[-i-1]
        ipower = 10**i
        carry = 0
        inner_res = 0
        for j in range(len(n2list)):
            n2dig = n2list[-j-1]
            raw_prod = (n2dig*n1dig) + carry
            carry = raw_prod//10
            prod = raw_prod%10
            inner_res += ((10**j) * prod)
        inner_res += ((10**(j+1)) * carry)
        final_res += ((10**i) * inner_res)
        
    return final_res

In [6]:
grade_school_mult(x,y)

7006652

### Karatsuba's Algorithm

In [7]:
def karatsuba(x,y):
    if x<10 and y<10:
        return x*y
    if x>y:
        max_length = len(str(x))
        max_num = x
        min_num = y
    else:
        max_length = len(str(y))
        max_num = y
        min_num = x
    if max_length%2==0:
        first_multiplier = max_length
    else:
        first_multiplier = max_length - 1
    div_length = max_length//2
    a = x//(10**div_length)
    b = x%(10**div_length)
    c = y//(10**div_length)
    d = y%(10**div_length)
    ac = karatsuba(a,c)
    bd = karatsuba(b,d)
    sumterm = karatsuba(a+b,c+d)
    leftoversum = sumterm - ac - bd
    
    product = (10**first_multiplier*ac) + (10**div_length*leftoversum) + bd
    return product

In [8]:
karatsuba(x,y)

7006652

In [9]:
%%time
#https://www.coursera.org/learn/algorithms-divide-conquer/exam/srsxO/programming-assignment-1/attempt
n1 = 3141592653589793238462643383279502884197169399375105820974944592
n2 = 2718281828459045235360287471352662497757247093699959574966967627
karatsuba(n1,n2)

Wall time: 3.89 ms


8539734222673567065463550869546574495034888535765114961879601127067743044893204848617875072216249073013374895871952806582723184

# 2. Merge Sort

In [10]:
inp = [57,42,63,32,130,6,23,45]

In [11]:
sorted(inp)

[6, 23, 32, 42, 45, 57, 63, 130]

In [12]:
def merge_sort(arr2):
    arr = copy.deepcopy(arr2)
    n = len(arr)
    split = n//2
    #Base Case
    if n==1:
        return arr
    
    #Split
    left = arr[:split]
    right = arr[split:]
    leftsort = merge_sort(left)
    rightsort = merge_sort(right)
    #Merge
    merged_arr = []
    i=0
    j=0
    while i<len(leftsort) and j<len(rightsort):
        if leftsort[i]<rightsort[j]:
            merged_arr.append(leftsort[i])
            i+=1
        else:
            merged_arr.append(rightsort[j])
            j+=1
    #Check for leftover elements
    while i<len(leftsort):
        merged_arr.append(leftsort[i])
        i+=1
    while j<len(rightsort):
        merged_arr.append(rightsort[j])
        j+=1
    return merged_arr

In [13]:
inp

[57, 42, 63, 32, 130, 6, 23, 45]

In [14]:
merge_sort(inp)

[6, 23, 32, 42, 45, 57, 63, 130]

# 3. Counting Array Inversions

### Double Loop (Brute Force)

In [15]:
def bruteforce_count_inversions(arr2):
    #Brute Force Method
    arr = arr2.copy()
    tot_inversions = 0
#     inversion_list = []
    for i in range(len(arr)):
        for j in range(i,len(arr)):
            if arr[i]>arr[j]:
#                 inversion_list.append((arr[i],arr[j]))
                tot_inversions+=1
    return tot_inversions
#     return tot_inversions, inversion_list

In [16]:
bruteforce_count_inversions([1,3,5,2,4,6])

3

In [17]:
bruteforce_count_inversions([8,4,2,1])

6

### Divide & Conquer (Merge Sort Based)

High Level Algorithm:
```python
def count_inversions(arr2):
    arr = arr2.copy()
    n = len(arr)
    split = n//2
    if n==1:
        return 0
    left_inv = count_inversions(arr[:split])
    right_inv = count_inversions(arr[split:])
    split_inv = count_split_inversions(arr)
    tot_inversions = left_inv + right_inv + split_inv
    return tot_inversions
```

We wanna implement count_split_inversions in linear time. For this, we'll piggyback on the merge sort.

In [18]:
def sort_and_count_inv(arr2):
    '''
    Returns sorted array, number of inversions
    '''
    arr = copy.deepcopy(arr2)
    n = len(arr)
    split = n//2
    #Base Case
    if n==1:
        return arr, 0
    
    #Split and count left and right inversions
    left = arr[:split]
    right = arr[split:]
    leftsort,left_inv = sort_and_count_inv(left)
    rightsort,right_inv = sort_and_count_inv(right)
    
    #Counting split inversions: While Merging
    split_inv = 0
    merged_arr = []
    i=0
    j=0
    while i<len(leftsort) and j<len(rightsort):
        if leftsort[i]<rightsort[j]:
            merged_arr.append(leftsort[i])
            i+=1
        else:
            merged_arr.append(rightsort[j])
            split_inv = split_inv + (len(leftsort) - i)         #The right quantity is the number of elements in leftsort
#                                                                    that have not been added to merged_arr yet.
            j+=1
    #Check for leftover elements
    while i<len(leftsort):
        merged_arr.append(leftsort[i])
        i+=1
    while j<len(rightsort):
        merged_arr.append(rightsort[j])
        j+=1
        
    tot_inv = left_inv + right_inv + split_inv
    return merged_arr, tot_inv

In [19]:
sort_and_count_inv([1,3,5,2,4,6])[1]

3

In [20]:
sort_and_count_inv([8,4,2,1])[1]

6

In [21]:
print("Let's compare the times for a random array of length 10000")
k = list(np.random.rand(10))
# k = list(np.random.rand(10000))


t1 = process_time()
ans1 = sort_and_count_inv(k)[1]
t2 = process_time()
print(f"Answer 1 = {ans1}, Time 1 = {t2-t1}s")

t3 = process_time()
ans2 = bruteforce_count_inversions(k)
t4 = process_time()
print(f"Answer 2 = {ans2}, Time 2 = {t4-t3}s")

Let's compare the times for a random array of length 10000
Answer 1 = 24, Time 1 = 0.0s
Answer 2 = 24, Time 2 = 0.0s


In [22]:
# #https://www.coursera.org/learn/algorithms-divide-conquer/exam/YLbzP/programming-assignment-2/attempt
# with open(r"C:\Users\Sid\Desktop\array_list.txt","r") as fp:
#     arr = np.loadtxt(fp)
#     print(sort_and_count_inv(arr)[1])

# #Answer is 2407905288

# 4. Matrix Multiplication

We'll assume the matrices are square (n $\times$ n matrices), but all of the below can be generalised for n $ \times $ m as well.

Further, we have not used numpy. We could use that, and the code would become significantly shorter. But, the main reason for not using numpy is consistency - if we were to use numpy, why not just use np.dot instead? Thus we've worked with the default python lists.

### Definition Based

In [23]:
arr1 = [[1, 2, 3,4], [5, 6,7,8],[9,10,11,12],[13,14,15,16]]
arr2 = [[3,4,1,6], [9,2,5,7],[1,10,3,7],[9,1,2,1]]

In [24]:
def standard_matmul(A,B):
    n = len(A)
    assert len(B)==n
    
    C = copy.deepcopy(A)
    for i in range(n):
        for j in range(n):
            C[i][j] = 0
            for k in range(n):
                C[i][j] += A[i][k]*B[k][j]
    return C

In [25]:
np.dot(arr1,arr2)

array([[ 60,  42,  28,  45],
       [148, 110,  72, 129],
       [236, 178, 116, 213],
       [324, 246, 160, 297]])

In [26]:
standard_matmul(arr1,arr2)

[[60, 42, 28, 45],
 [148, 110, 72, 129],
 [236, 178, 116, 213],
 [324, 246, 160, 297]]

### Simple Recursion: 8 Recursive Calls

In [27]:
def split_arr(arr):
    A,B,C,D = [],[],[],[]
    n=len(arr)
    for i in range(n):
        templistA,templistB,templistC,templistD = [],[],[],[]
        if i<n//2:
            for j in range(n):
                    if j<n//2:
                        templistA.append(arr[i][j])
                    else:
                        templistB.append(arr[i][j])
            A.append(templistA)
            B.append(templistB)
        else:
            for j in range(n):
                    if j<n//2:
                        templistC.append(arr[i][j])
                    else:
                        templistD.append(arr[i][j])
            C.append(templistC)
            D.append(templistD)
    return A,B,C,D

def merge_arr(A,B,C,D):
    if type(A)==int and type(B)==int and type(C)==int and type(D)==int:
        return [[A,B],[C,D]]
    
    midn = len(A)
    n = 2*midn
    arr = [[0 for j in range(0, n)] for i in range(0, n)]
    
    for i in range(0, midn):
        for j in range(0, midn):
            arr[i][j] = A[i][j]
            arr[i][j + midn] = B[i][j]
            arr[i + midn][j] = C[i][j]
            arr[i + midn][j + midn] = D[i][j]
    return arr


def add_matrix(X,Y):
    if type(X)==int and type(Y)==int:
        return X+Y
    assert len(X)==len(Y)
    n = len(X)
    
    Z = copy.deepcopy(X)
    
    for i in range(n):
        for j in range(n):
            Z[i][j] = X[i][j] + Y[i][j]
    return Z

def sub_matrix(X,Y):
    if type(X)==int and type(Y)==int:
        return X+Y
    assert len(X)==len(Y)
    n = len(X)
    
    Z = copy.deepcopy(X)
    
    for i in range(n):
        for j in range(n):
            Z[i][j] = X[i][j] - Y[i][j]
    return Z

In [28]:
def rec_matmul(X,Y):
    assert len(X)==len(Y)
    n = len(X)
    if n==1:
        return X[0][0]*Y[0][0]
    
    A,B,C,D = split_arr(X)
    E,F,G,H = split_arr(Y)
    
    AE = rec_matmul(A,E)
    BG = rec_matmul(B,G)
    AF = rec_matmul(A,F)
    BH = rec_matmul(B,H)
    CE = rec_matmul(C,E)
    DG = rec_matmul(D,G)
    CF = rec_matmul(C,F)
    DH = rec_matmul(D,H)

    L1 = add_matrix(AE, BG)
    L2 = add_matrix(AF, BH)
    L3 = add_matrix(CE, DG)
    L4 = add_matrix(CF, DH)

    fin_arr = merge_arr(L1,L2,L3,L4)
    
    return fin_arr

In [29]:
rec_matmul(arr1,arr2)

[[60, 42, 28, 45],
 [148, 110, 72, 129],
 [236, 178, 116, 213],
 [324, 246, 160, 297]]

### Strassen's Algorithm

In [30]:
def strassen(X,Y):
    assert len(X)==len(Y)
    n = len(X)
    if n==1:
        return X[0][0]*Y[0][0]
    
    A,B,C,D = split_arr(X)
    E,F,G,H = split_arr(Y)
    
    P1 = rec_matmul(A,sub_matrix(F,H))
    P2 = rec_matmul(add_matrix(A,B),H)
    P3 = rec_matmul(add_matrix(C,D),E)
    P4 = rec_matmul(D,sub_matrix(G,E))
    P5 = rec_matmul(add_matrix(A,D),add_matrix(E,H))
    P6 = rec_matmul(sub_matrix(B,D),add_matrix(G,H))
    P7 = rec_matmul(sub_matrix(A,C),add_matrix(E,F))

    L1 = sub_matrix(add_matrix(add_matrix(P4, P5),P6),P2)
    L2 = add_matrix(P1, P2)
    L3 = add_matrix(P3, P4)
    L4 = sub_matrix(add_matrix(P1, P5),add_matrix(P3, P7))

    fin_arr = merge_arr(L1,L2,L3,L4)
    
    return fin_arr

In [31]:
strassen(arr1,arr2)

[[60, 42, 28, 45],
 [148, 110, 72, 129],
 [236, 178, 116, 213],
 [324, 246, 160, 297]]

# 5. Closest Pairs

## 1D Case

We want to find the pair of points which are located closest to each other, in the whole set. Here, the pts are 1D and thus each point has 1 coordinate associated with it

### Brute Force

In [32]:
given_pts1d = [1.34, 3.84, 9.72, 10.69, 6.81, 14.2]

In [33]:
def brute_closestpair1d(arr):
    mindist = abs(arr[1] - arr[0]) #The euclidean dist.
    minpts = [arr[0],arr[1]]   #The 2 pts which are closest.
    for i in range(len(arr)):
        for j in range(i+1,len(arr)):
            dist = abs(arr[i]-arr[j])
            if dist<mindist:
                minpts=[arr[i],arr[j]]
                mindist = dist
    return minpts, mindist            #minpts is of form [pt1, pt2]

In [34]:
brute_closestpair1d(given_pts1d)

([9.72, 10.69], 0.9699999999999989)

In [35]:
10.69 - 9.72

0.9699999999999989

This algorithm has $\mathcal{O}(n^2)$ time complexity. We can do better by sorting it.

### Using Merge Sort

If we sort them, using merge-sort, we take $\mathcal{O}(n \log{n} )$ time . Then, finding the closest pair can be done in $\mathcal{O}(n)$ time as follows. 

In [36]:
def closestpair1d(arr2):
    arr = merge_sort(arr2)
    mindist = abs(arr[1] - arr[0]) #The euclidean dist.
    minpts = [arr[0],arr[1]]   #The 2 pts which are closest.
    for i in range(len(arr)-1):
        j = i+1
        dist = abs(arr[i]-arr[j])
        if dist<mindist:
            minpts=[arr[i],arr[j]]
            mindist = dist
    return minpts, mindist            #minpts is of form [pt1, pt2]

In [37]:
closestpair1d(given_pts1d)

([9.72, 10.69], 0.9699999999999989)

## 2D Case

We are given n points in the plane and you want to figure out which pair of points are closest to each other.

### Brute Force

This algorithm has $\mathcal{O}(n^2)$ time complexity.

In [38]:
given_pts2d = [[2,3], [1,1], [0,0], [3,5], [1,4], [1,0.5]]

In [39]:
def dist_2d(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    dist = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return dist

In [40]:
def brute_closestpair2d(arr):
    mindist = dist_2d(arr[0],arr[1]) #The euclidean dist.
    minpts = [arr[0],arr[1]]   #The 2 pts which are closest.
    for i in range(len(arr)):
        for j in range(i+1,len(arr)):
            dist = dist_2d(arr[j],arr[i])
            if dist<mindist:
                minpts=[arr[j],arr[i]]
                mindist = dist
    return minpts, mindist            #minpts is of form [pt1, pt2]

In [41]:
brute_closestpair2d(given_pts2d)

([[1, 0.5], [1, 1]], 0.5)

### Recursive

As in the 1D case, sorting it might be beneficial. Let's try a basic recursive approach. We'll sort in X and Y both (i.e 2 different sorts)

In [42]:
def merge_sort_2darr(arr2,sort_index):
    assert sort_index==0 or sort_index==1
    # Sort index can be 0 or 1, i.e x and y coordinate on whose basis the array is to be sorted.
    arr = copy.deepcopy(arr2)
    n = len(arr)
    split = n//2
    #Base Case
    if n==1:
        return arr
    
    #Split
    left = arr[:split]
    right = arr[split:]
    leftsort = merge_sort_2darr(left, sort_index)
    rightsort = merge_sort_2darr(right, sort_index)
    #Merge
    merged_arr = []
    i=0
    j=0
    while i<len(leftsort) and j<len(rightsort):
        if leftsort[i][sort_index]<rightsort[j][sort_index]:
            merged_arr.append(leftsort[i])
            i+=1
        else:
            merged_arr.append(rightsort[j])
            j+=1
    #Check for leftover elements
    while i<len(leftsort):
        merged_arr.append(leftsort[i])
        i+=1
    while j<len(rightsort):
        merged_arr.append(rightsort[j])
        j+=1
    return merged_arr


In [43]:
def closest_pair(Px,Py):
    '''Inputs are:
    Px: Sorted arr in x coordinate
    Py: Sorted arr in y coordinate
    '''
    n = len(Px)
    #Base case: Brute force for n=2or 3
    if n<=3:
        return brute_closestpair2d(Px)
        
    #End of base case
    
    #Let Q = left half and R = right half
    Qx = Px[:n//2]
    Rx = Px[n//2:]
    
    Qy = Py[:n//2]
    Ry = Py[n//2:]
    
    #Lucky case: The closest pair is not split.
    minpts1,d1 = closest_pair(Qx, Qy)
    minpts2,d2 = closest_pair(Rx, Ry)
    
    delta = min(d1,d2)
    
    #Unlucky case: The closes pair is split, i.e, one pt is in left half, other in right half
    minpts3,d3 = closest_split_pair(Px, Py, delta)
    
    mindist = min(d1,d2,d3)
    
    if mindist == d1:
        return minpts1,d1
    elif mindist == d2:
        return minpts2,d2
    else:
        assert minpts3 is not None
        return minpts3,d3


In [44]:
def closest_split_pair(Px, Py, delta):
    n = len(Px)
    xbar = Px[n//2][0]
    Sy = []
    for pt in Px:
        if xbar - delta <= pt[0] <= xbar + delta :
            Sy.append(pt)
    mindist = delta
    minpts = None 

    Sy = merge_sort_2darr(Sy,1)
    for i in range(len(Sy)):
        for j in range(i+1, len(Sy)): 
            if Sy[j][1] - Sy[i][1] > delta:
                break
            p = Sy[i] 
            q = Sy[j] 
            dist = dist_2d(p, q) 
            if dist < mindist:
                mindist = dist 
                minpts = [p,q]
    
    return minpts, mindist


In [45]:
def final_solve_closestpairs(arr2):
    arr = copy.deepcopy(arr2)
    Px = merge_sort_2darr(arr,0)
    Py = merge_sort_2darr(arr,1)
    return closest_pair(Px, Py)

In [46]:
final_solve_closestpairs(given_pts2d)

([[1, 0.5], [1, 1]], 0.5)